In [7]:
import pandas as pd
import requests
import json
import threading
import time
from multiprocessing import Pool, Manager
from itertools import repeat

In [2]:
df = pd.DataFrame()

In [3]:
# threading을 활용하기 위해서 직방 크롤링을 하는 함수를 만들었다.
def zigbang_seoul_crawler(i):
    time.sleep(1)
    # 위에서 만든 전역변수 df를 활용하기 위해서 이런 코드를 썼다. 맞는지는 잘 모르겠다.
    global df
    # 상품번호가 비어있어서 아무 자료도 읽히지 않을 경우를 대비해서 try - except문을 활용했고
    # 예외상황이 발생할 경우 그냥 지나가도록 pass 하도록 했다.
    try:
        # 파라미터 i를 통해서 상품번호를 전달하고 그것을 api url과 합쳐서 개별 상품들의 정보에 접근했다.
        url = "https://apis.zigbang.com/v2/items/" + str(i)
        # 따로 header같은것 필요없이 제대로 response 200이 뜨고 자료도 읽혀서 이대로 진행했다
        req = requests.get(url) 
        data = json.loads(req.text)
        # 개별 매물들의 데이터 정리
        item = data['item']
        si = item['local1']
        gu = item['local2']
        dong = item['local3']
        jibun = item['jibunAddress']
        manage_cost = item["manage_cost"]
        loc = item['random_location']
        sales_type = item["sales_type"]
        floor = item['floor']
        m2 = item['공급면적_m2']
        rent = item['월세금액']
        deposit = item['보증금액']
        near_subway = data['subways']
        service_type = item["service_type"]
        # 읽어온 매물 데이터에서 서울시의 월세, 전세 매물의 데이터를 Series 형태로 df에 append 했다.
        if si == '서울시' and (sales_type == "월세" or sales_type =="전세"):
            x = pd.Series([si,gu,dong,jibun,sales_type,service_type,floor,m2,rent,deposit,manage_cost,loc,near_subway])
            df = df.append(x, ignore_index = True)
        else:
        # 서울시면서 월세 또는 전세 데이터가 아닌경우 df에 따로 저장하지 않고 pass 했다.
            pass
        print(i)
    except:
        pass

In [ ]:
# 크롤링 도중 과도한 requests로 인해서 ip차단당하는것을 막기 위해서 1만번씩 크롤링을 진행하고 
# 10초를 쉬고 다음 1만개를 다시 진행하는 형태로 했다. 이것을 60번 반복했다. 
# 또한 60만회의 크롤링을 빠르게 진행하기 위해서 threading을 사용했고 그 결과 엄청난 시간 단축을 할 수 있었다.
# 60만개를 다 크롤링하는데 약 1시간정도 걸렸다.
for i in range(1,61):
    for j in range(31000000 + (10000 * (i - 1)), 31000000 + (10000 * i)):
        th = threading.Thread(target = zigbang_seoul_crawler, args = (j,))
        th.start()
    time.sleep(10)
    # 중간에 ip차단으로 크롤링이 끊기는것을 방지하기 위해 1만개마다 csv파일을 저장했다.
    df.to_csv(f"zigbang_seoul{i}.csv")

    # 만약 매물번호가 지역별로 코드가 정해져있었다면 이런식으로 60만개의 데이터를 모두 크롤링 할 필요는 없었을것같다
# 하지만 확인 결과 지역에 따른 매물번호가 따로 지정되어있지 않았기때문에 모두 긁어온 후 그 중 조건에 맞는 자료만을
# 저장하는 형태로 진행했다.

In [ ]:
# 60개의 zigbang_seoul.csv 파일명을 list형태로 만든다
lst = []
for i in range(1,61):
    lst.append(f"zigbang_seoul{i}.csv")

In [ ]:
# df에  60개의 csv자료를 모두 합친다.
df = pd.concat(
    map(pd.read_csv, lst), ignore_index=True)

In [ ]:
df_one = df.loc[df['5'] == "원룸"]

In [5]:
def fuck_you(i):
    return i

In [ ]:
if __name__=='__main__':
    start_time = time.time()
    pool = Pool(processes=4) # 4개의 프로세스를 사용합니다.
    pool.map(fuck_you, range(1,100)) # get_contetn 함수를 넣어줍시다.
    print("--- %s seconds ---" % (time.time() - start_time))